<a href="https://colab.research.google.com/github/chaeeun0405/ML_Class/blob/main/6%EC%A3%BC%EC%B0%A8/ML_6-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

# ========================================
# 경로 설정 (여기를 수정하세요)
# ========================================
DATA_DIR = '/content/drive/MyDrive/dataset'  # CIFAR-10 데이터를 저장할 경로

In [2]:
# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

In [3]:
# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')

if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")

데이터가 없습니다. /content/drive/MyDrive/dataset에 다운로드합니다...


In [4]:
# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}\n")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)



In [5]:
# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [6]:
# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

In [7]:
# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



In [8]:
# 선택된 고양이, 개, 말 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[int(y)]for y in y_train])
y_test = np.array([label_map[int(y)]for y in y_test])

/tmp/ipython-input-705246257.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_train = np.array([label_map[int(y)]for y in y_train])
/tmp/ipython-input-705246257.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test = np.array([label_map[int(y)]for y in y_test])


In [9]:
# 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

In [10]:
print("x_train shape:", x_train.shape)  # (15000, 32, 32, 3)
print("x_test shape:", x_test.shape)    # (3000, 32, 32, 3)

x_train shape: (15000, 32, 32, 3)
x_test shape: (3000, 32, 32, 3)


In [11]:
# CNN 모델 구성
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# 학습
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

Epoch 1/10
106/106 - 9s - 87ms/step - accuracy: 0.5077 - loss: 0.9488 - val_accuracy: 0.6020 - val_loss: 0.8545
Epoch 2/10
106/106 - 1s - 6ms/step - accuracy: 0.6220 - loss: 0.7938 - val_accuracy: 0.6267 - val_loss: 0.7712
Epoch 3/10
106/106 - 1s - 7ms/step - accuracy: 0.6744 - loss: 0.7079 - val_accuracy: 0.6720 - val_loss: 0.7261
Epoch 4/10
106/106 - 1s - 8ms/step - accuracy: 0.7117 - loss: 0.6508 - val_accuracy: 0.6853 - val_loss: 0.7083
Epoch 5/10
106/106 - 1s - 7ms/step - accuracy: 0.7313 - loss: 0.6147 - val_accuracy: 0.7247 - val_loss: 0.6366
Epoch 6/10
106/106 - 1s - 7ms/step - accuracy: 0.7453 - loss: 0.5820 - val_accuracy: 0.7393 - val_loss: 0.5963
Epoch 7/10
106/106 - 1s - 7ms/step - accuracy: 0.7670 - loss: 0.5403 - val_accuracy: 0.7500 - val_loss: 0.5921
Epoch 8/10
106/106 - 1s - 7ms/step - accuracy: 0.7751 - loss: 0.5262 - val_accuracy: 0.7527 - val_loss: 0.5714
Epoch 9/10
106/106 - 1s - 8ms/step - accuracy: 0.7927 - loss: 0.4900 - val_accuracy: 0.7540 - val_loss: 0.5897


In [14]:
# 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

94/94 - 1s - 14ms/step - accuracy: 0.7570 - loss: 0.5699

✅ Test Accuracy: 0.7570


In [15]:
# 예측 예시
predictions = model.predict(x_test[:5])
print("예측 결과:", np.argmax(predictions, axis=1))
print("실제 레이블:", y_test[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
예측 결과: [0 0 0 2 1]
실제 레이블: [0 0 1 2 1]
